In [22]:
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli
import numpy as np

In [ ]:
?AerSimulator.

Init signature:
AerSimulator(
    configuration=None,
    properties=None,
    provider=None,
    target=None,
    **backend_options,
)
Docstring:     
Noisy quantum circuit simulator backend.

**Configurable Options**

The `AerSimulator` supports multiple simulation methods and
configurable options for each simulation method. These may be set using the
appropriate kwargs during initialization. They can also be set of updated
using the :meth:`set_options` method.

Run-time options may also be specified as kwargs using the :meth:`run` method.
These will not be stored in the backend and will only apply to that execution.
They will also override any previously set options.

For example, to configure a density matrix simulator with a custom noise
model to use for every execution

.. code-block:: python

    noise_model = NoiseModel.from_backend(backend)
    backend = AerSimulator(method='density_matrix',
                            noise_model=noise_model)

**Simulating an IBM Quantum Back

In [ ]:


def build_cost_layer(qc, h, w, gamma):
    """Apply e^{-i * gamma * H_cost}"""
    n = len(h)
    # Linear terms: h_i Z_i
    for i in range(n):
        if h[i] != 0:
            qc.rz(2 * gamma * h[i], i)

    # Quadratic terms: w_ij Z_i Z_j
    for (i, j), wij in w.items():
        if wij != 0:
            qc.cx(i, j)
            qc.rz(2 * gamma * wij, j)
            qc.cx(i, j)


def build_mixer_layer(qc, beta):
    """Apply e^{-i * beta * H_mix} where H_mix = sum X_i"""
    for q in range(qc.num_qubits):
        qc.rx(2 * beta, q)


def qaoa_run(h, w, gammas, betas):
    """
    h: list of linear coefficients
    w: dict with keys (i,j) for quadratic coefficients
    gammas, betas: lists of length p
    """
    p = len(gammas)
    n = len(h)

    # Step 1: initialize |+>^n
    qc = QuantumCircuit(n)
    qc.h(range(n))

    # Step 2: QAOA layers
    for gamma, beta in zip(gammas, betas):
        build_cost_layer(qc, h, w, gamma)
        build_mixer_layer(qc, beta)

    qc.measure_all()

    # Step 3: run on simulator
    simulator = AerSimulator()
    # result = execute(qc, backend, shots=2048).result()
    simulator.run()
    counts = result.get_counts()

    # Step 4: return optimal bitstring (max probability)
    best_bitstring = max(counts, key=counts.get)
    return best_bitstring, counts


In [20]:

def qaoa_with_angles(qp: QuadraticProgram, gammas, betas):
    p = len(gammas)
    reps = p

    # Put all angles into the single array expected by QAOA
    # Order: [γ1,...,γp, β1,...,βp]
    params = gammas + betas

    qaoa = QAOA(sampler=Sampler(), reps=reps, initial_point=params)
    solver = MinimumEigenOptimizer(qaoa)

    result = solver.solve(qp)
    return result.x  # optimal bitstring


In [21]:
# Example: MaxCut on a 3‑node line graph
h = [0, 0, 0]
w = {(0,1): 1, (1,2): 1}

gammas = [0.8, 1.2]
betas  = [0.4, 0.9]

best, counts = qaoa_run(h, w, gammas, betas)
print("Best bitstring:", best)


NameError: name 'AerSimulator' is not defined